In [ ]:
%load_ext autoreload
%autoreload 2
import time
import timeit

from biometric_system_euclidiano import BiometricSystem

from anomaly_detectors.M2005 import M2005 
from anomaly_detectors import thresholds
from data_stream import data_stream
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold

from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import kneighbors_graph
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import networkx as nx


import json
import ipdb
import os, sys
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
import random
import copy
import IPython.display as ipd


def Average(lst):
    x = sum(lst)
    y = len(lst)
    a = (round(x,6) / round(y))
    return a


def split_data_enrollment(dataset, column, n_samples):
    data_to_enrollment = dict()

    for value in dataset[column].unique():
        
        data_to_enrollment.setdefault(value, dataset.loc[dataset[column]==value].iloc[:(n_samples//2)].loc[:,~dataset.columns.isin([column])].reset_index(drop=True))
    
    return data_to_enrollment

def split_data_validation(dataset, column, n_samples):
    data_to_validation = dict()

    for value in dataset[column].unique():
        
        data_to_validation.setdefault(value, dataset.loc[dataset[column]==value].iloc[(n_samples//2):n_samples].loc[:,~dataset.columns.isin([column])].reset_index(drop=True))
    
    return data_to_validation


def split_data_recognition(dataset, column, n_samples):
    data_to_recognition = dict()

    for value in dataset[column].unique():
 
        data_to_recognition.setdefault(value, dataset.loc[dataset[column]==value].iloc[:n_samples].loc[:,~dataset.columns.isin([column])].reset_index(drop=True))
    
    
    return data_to_recognition

def split_data(dataset, column, n_samples):
    data_to_enrollment = dict()
    data_to_validation = dict()
    data_to_recognition = dict()

    for value in dataset[column].unique():
        
        #Treinamento, pegando dados de usuarios 
        data_to_enrollment.setdefault(value, dataset.loc[dataset[column]==value].iloc[:(n_samples//2)].loc[:,~dataset.columns.isin([column])].reset_index(drop=True))

        # Definir o limiar de decisao dos algoritmos de classificacao
        data_to_validation.setdefault(value, dataset.loc[dataset[column]==value].iloc[(n_samples//2):n_samples].loc[:,~dataset.columns.isin([column])].reset_index(drop=True))

        # Criar o fluxo de teste
        data_to_recognition.setdefault(value, dataset.loc[dataset[column]==value].iloc[n_samples:].loc[:,~dataset.columns.isin([column])].reset_index(drop=True))
    
    
    return data_to_enrollment,data_to_validation, data_to_recognition

#--------------------------------------------------------------------------------------------------------------# 
# INICIO DO TIMER
inicio = timeit.default_timer()
#--------------------------------------------------------------------------------------------------------------# 


df = pd.read_csv('dados/DSL-Modificado.csv', delimiter = ',', index_col = [0])
df = df.drop(['rep'], axis=1)
users = df['subject'].unique()

perc = 0.5
impostor_rate = 0.30
rate_external_impostor = 0
R=1
GRAPH_MIN_CUT_GROWING = []

#--------------------------------------------------------------------------------------------------------------# 
#Separação de index, primeiro pra treino, segundo para validação e teste
# Dependendo do teste é necessario trocar a linha ("Separação por Index")

sessionIndex1 = 1

#--------------------------------------------------------------------------------------------------------------# 
#Separação de usuarios

len_reg_users = int(len(users) * perc)

kfold = KFold(n_splits=2, shuffle=True, random_state=R)
splits = kfold.split(users)

#--------------------------------------------------------------------------------------------------------------# 
#Registro de usuarios

for i, (reg_users, not_reg_users) in enumerate(splits):

    internal_users = copy.deepcopy(df.loc[df['subject'].isin(users[reg_users])])
    external_users = copy.deepcopy(df.loc[~df['subject'].isin(users[reg_users])])
   
    frames = [internal_users, external_users]

    internal_users = pd.concat(frames)
#--------------------------------------------------------------------------------------------------------------#  
#Separação por Index

dataS1 = internal_users.loc[(internal_users['sessionIndex'] == sessionIndex1)]
dataS1.drop(["sessionIndex"], axis=1, inplace=True)

dataS2 = internal_users.loc[(internal_users['sessionIndex'] != sessionIndex1)]
dataS2.drop(["sessionIndex"], axis=1, inplace=True)


#--------------------------------------------------------------------------------------------------------------#  
# Dados para Treino, Validação e Reconhecimento

#Treinamento, pegando dados de usuarios 
data_to_enrollment = split_data_enrollment(dataS1, column='subject', n_samples=50)

# Definir o limiar de decisao dos algoritmos de classificacao
data_to_validation = split_data_validation(dataS1, column='subject', n_samples=50)

# Criar o fluxo de teste
data_to_recognition = split_data_recognition(dataS2, column='subject', n_samples=350)

_, _, external_users_data = split_data(external_users, column='subject', n_samples=50)


#--------------------------------------------------------------------------------------------------------------# 

#Sistema com Adaptação (GraphMinCut)

detector = M2005()
adaptive= "GrowingWindow"
system = BiometricSystem(detector=detector, random_state=R)
system.enrollment_grafos(dataset=data_to_enrollment, adaptive=adaptive)

decision_threshold = thresholds.best_threshold(data_to_validation, system, size=10, random_state=R)

metrics_adaptativo_grafo_growing = dict()

lista_nao_usadas_grafo_growing = list()
lista_usadas_grafo_growing = list()

auxiliar_euclidiano = {}

for j, genuine in enumerate(system.users.keys()):


    ipd.clear_output(wait=True)
    print(f"Rodando GraphMinCutGrowing")
    print(f"Testando usuário {j+1}/{len(system.users.keys())}")

    datastream = data_stream.Random(impostor_rate= impostor_rate,
                                    rate_external_impostor=rate_external_impostor,
                                    random_state=R)

    test_stream, y_true, amostras_grafo_growing_genuinas,amostras_grafo_growing_impostoras = datastream.create(genuine,
                                                                                                data_to_recognition,
                                                                                                external_users_data)



    y_pred, lista_nao_usadas_grafo_growing2, lista_usadas_grafo_growing2 = system.autenticate_grafos(genuine,
                                                                                test_stream,
                                                                                decision_threshold=decision_threshold,
                                                                                adaptive_TESTE=adaptive,
                                                                                return_scores=False)

    lista_nao_usadas_grafo_growing.append(lista_nao_usadas_grafo_growing2)
    lista_usadas_grafo_growing.append(lista_usadas_grafo_growing2)

    fmr, fnmr, b_acc,  y_genuine , y_impostor = system.compute_metrics(y_true, y_pred)
    auxiliar_euclidiano = y_genuine.value_counts()[1], y_impostor.value_counts()[1]

    for met in ['fmr','fnmr','b_acc']:
        metrics_adaptativo_grafo_growing.setdefault(genuine, dict()).setdefault(met,[]).append(eval(met))
    #json.dump(metrics_adaptativo_grafo_growing, open("metricas_grafos_growing.json", "w"))


usuarios = metrics_adaptativo_grafo_growing.keys()
result = pd.DataFrame(metrics_adaptativo_grafo_growing.values())

fmr_mean = []
fnmr_mean = []
b_acc_mean = []

for i in result['fmr']:
    fmr_mean.append(Average(i))

for i in result['fnmr']:
    fnmr_mean.append(Average(i))

for i in result['b_acc']:
    b_acc_mean.append(Average(i))

metrics_adaptativo_grafo_growing_mean = pd.DataFrame(list(zip(usuarios, fmr_mean, fnmr_mean,b_acc_mean)),
            columns =['Usuarios','fmr_mean', 'fnmr_mean','b_acc_mean'])

GRAPH_MIN_CUT_GROWING.append(metrics_adaptativo_grafo_growing_mean['b_acc_mean'].mean())

#--------------------------------------------------------------------------------------------------------------#     

fim = timeit.default_timer()
horas, rem = divmod(fim-inicio, 3600)
minutos, segundos = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(horas),int(minutos),segundos))

print('---------------------')
print("Teste com Grafos distancia Euclidiana")
print("GRAPH_MIN_CUT_GROWING", GRAPH_MIN_CUT_GROWING)



#--------------------------------------------------------------------------------------------------------------# 

Rodando GraphMinCutGrowing
Testando usuário 1/51
> c:\users\murilo\documents\mestrado\sistemas mestrado - copia\sistema com grafos(tentativa de proposta)\biometric_system_euclidiano.py(223)Distancia_euclidiana()
    221         import pdb; pdb.set_trace()
    222 
--> 223         E_dist = euclidean_distances(Y, X)
    224         #E_dist.max()
    225 

ipdb> E_dist 
*** NameError: name 'E_dist' is not defined
ipdb> Y
[[0.3085, 0.3531, 0.4553, 0.2063, 0.3326, 0.2769, 0.4237, 0.1858, 0.4244, 0.4347, 0.5258, 0.3333, 0.7432, 0.7194, 0.8208, 0.6418, 0.8913, 0.9056, 0.9832, 0.8137, 0.3487, 0.3475999999999999, 0.4395, 0.2568, 0.2574, 0.2864, 0.3772, 0.1666, 0.2782, 0.259, 0.3788, 0.1584, 0.6927, 0.6919, 0.7925, 0.5921, 0.4863, 0.4842, 0.584, 0.3865], [0.6347, 0.6231, 0.6682, 0.5896, 0.5865, 0.6000000000000001, 0.6335000000000001, 0.553, 0.9592, 0.9465, 0.9935, 0.9122, 1.1927, 1.1925, 1.2268, 1.1584, 0.5171, 0.5105, 0.5446, 0.483, 0.6828, 0.6909, 0.7183999999999999, 0.6553, 0.3793, 0.3896, 0.

ipdb> type(Y)
<class 'list'>
ipdb> len(Y)
26
ipdb> n
> c:\users\murilo\documents\mestrado\sistemas mestrado - copia\sistema com grafos(tentativa de proposta)\biometric_system_euclidiano.py(226)Distancia_euclidiana()
    224         #E_dist.max()
    225 
--> 226         result = np.where(E_dist == np.amax(E_dist))
    227 
    228         ponto1 = int(result[0])

ipdb> result 
*** NameError: name 'result' is not defined
ipdb> n
> c:\users\murilo\documents\mestrado\sistemas mestrado - copia\sistema com grafos(tentativa de proposta)\biometric_system_euclidiano.py(228)Distancia_euclidiana()
    226         result = np.where(E_dist == np.amax(E_dist))
    227 
--> 228         ponto1 = int(result[0])
    229         ponto2 = int(result[1])
    230 

ipdb> result 
(array([10], dtype=int64), array([18], dtype=int64))
ipdb> lista_de_amostras
*** NameError: name 'lista_de_amostras' is not defined
ipdb> E_dist
array([[1.31067712, 1.6921189 , 0.94746591, 1.44363263, 1.38733715,
        1.67135216

ipdb> len(E_dist)
26


In [6]:
from biometric_system_euclidiano import BiometricSystem

from anomaly_detectors.M2005 import M2005 
from anomaly_detectors import thresholds
from data_stream import data_stream
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold

from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import kneighbors_graph
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import networkx as nx


import json
import ipdb
import os, sys
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
import random
import copy
import IPython.display as ipd


def Average(lst):
    x = sum(lst)
    y = len(lst)
    a = (round(x,6) / round(y))
    return a


def split_data_enrollment(dataset, column, n_samples):
    data_to_enrollment = dict()

    for value in dataset[column].unique():
        
        data_to_enrollment.setdefault(value, dataset.loc[dataset[column]==value].iloc[:(n_samples//2)].loc[:,~dataset.columns.isin([column])].reset_index(drop=True))
    
    return data_to_enrollment

def split_data_validation(dataset, column, n_samples):
    data_to_validation = dict()

    for value in dataset[column].unique():
        
        data_to_validation.setdefault(value, dataset.loc[dataset[column]==value].iloc[(n_samples//2):n_samples].loc[:,~dataset.columns.isin([column])].reset_index(drop=True))
    
    return data_to_validation


def split_data_recognition(dataset, column, n_samples):
    data_to_recognition = dict()

    for value in dataset[column].unique():
 
        data_to_recognition.setdefault(value, dataset.loc[dataset[column]==value].iloc[:n_samples].loc[:,~dataset.columns.isin([column])].reset_index(drop=True))
    
    
    return data_to_recognition

def split_data(dataset, column, n_samples):
    data_to_enrollment = dict()
    data_to_validation = dict()
    data_to_recognition = dict()

    for value in dataset[column].unique():
        
        #Treinamento, pegando dados de usuarios 
        data_to_enrollment.setdefault(value, dataset.loc[dataset[column]==value].iloc[:(n_samples//2)].loc[:,~dataset.columns.isin([column])].reset_index(drop=True))

        # Definir o limiar de decisao dos algoritmos de classificacao
        data_to_validation.setdefault(value, dataset.loc[dataset[column]==value].iloc[(n_samples//2):n_samples].loc[:,~dataset.columns.isin([column])].reset_index(drop=True))

        # Criar o fluxo de teste
        data_to_recognition.setdefault(value, dataset.loc[dataset[column]==value].iloc[n_samples:].loc[:,~dataset.columns.isin([column])].reset_index(drop=True))
    
    
    return data_to_enrollment,data_to_validation, data_to_recognition

#--------------------------------------------------------------------------------------------------------------# 
# INICIO DO TIMER
inicio = timeit.default_timer()
#--------------------------------------------------------------------------------------------------------------# 


df = pd.read_csv('dados/DSL-Modificado.csv', delimiter = ',', index_col = [0])
df = df.drop(['rep'], axis=1)
users = df['subject'].unique()

perc = 0.5
impostor_rate = 0.30
rate_external_impostor = 0
R=1
GRAPH_MIN_CUT_SLIDING = []

#--------------------------------------------------------------------------------------------------------------# 
#Separação de index, primeiro pra treino, segundo para validação e teste
# Dependendo do teste é necessario trocar a linha ("Separação por Index")

sessionIndex1 = 1
#--------------------------------------------------------------------------------------------------------------# 
#Separação de usuarios

len_reg_users = int(len(users) * perc)

kfold = KFold(n_splits=2, shuffle=True, random_state=R)
splits = kfold.split(users)

#--------------------------------------------------------------------------------------------------------------# 
#Registro de usuarios

for i, (reg_users, not_reg_users) in enumerate(splits):

    internal_users = copy.deepcopy(df.loc[df['subject'].isin(users[reg_users])])
    external_users = copy.deepcopy(df.loc[~df['subject'].isin(users[reg_users])])
    
    frames = [internal_users, external_users]

    internal_users = pd.concat(frames)

#--------------------------------------------------------------------------------------------------------------#  
#Separação por Index
dataS1 = internal_users.loc[(internal_users['sessionIndex'] == sessionIndex1)]
dataS1.drop(["sessionIndex"], axis=1, inplace=True)

dataS2 = internal_users.loc[(internal_users['sessionIndex'] != sessionIndex1)]
dataS2.drop(["sessionIndex"], axis=1, inplace=True)
#--------------------------------------------------------------------------------------------------------------#  
# Dados para Treino, Validação e Reconhecimento

#Treinamento, pegando dados de usuarios 
data_to_enrollment = split_data_enrollment(dataS1, column='subject', n_samples=50)

# Definir o limiar de decisao dos algoritmos de classificacao
data_to_validation = split_data_validation(dataS1, column='subject', n_samples=50)

# Criar o fluxo de teste
data_to_recognition = split_data_recognition(dataS2, column='subject', n_samples=350)

_, _, external_users_data = split_data(external_users, column='subject', n_samples=50)



 #Sistema com Adaptação (GraphMinCut)

detector = M2005()
adaptive = "SlidingWindow"
system = BiometricSystem(detector=detector, random_state=R)
system.enrollment_grafos(dataset=data_to_enrollment, adaptive=adaptive)

decision_threshold = thresholds.best_threshold(data_to_validation, system, size=10, random_state=R)

metrics_adaptativo_grafo_sliding = dict()

lista_nao_usadas_grafo_sliding = list()
lista_usadas_grafo_sliding = list()

auxiliar_euclidiano2 = {}

for j, genuine in enumerate(system.users.keys()):


    ipd.clear_output(wait=True)
    print(f"Rodando GraphMinCutSliding Euclidiano")
    print(f"Testando usuário {j+1}/{len(system.users.keys())}")

    datastream = data_stream.Random(impostor_rate= impostor_rate,
                                    rate_external_impostor=rate_external_impostor,
                                    random_state=R)

    test_stream, y_true, amostras_grafo_sliding_genuinas,amostras_grafo_sliding_impostoras = datastream.create(genuine,
                                                                                                data_to_recognition,
                                                                                                external_users_data)



    y_pred, lista_nao_usadas_grafo_sliding2, lista_usadas_grafo_sliding2 = system.autenticate_grafos(genuine,
                                                                                test_stream,
                                                                                decision_threshold=decision_threshold,
                                                                                adaptive_TESTE=adaptive,
                                                                                return_scores=False)

    lista_nao_usadas_grafo_sliding.append(lista_nao_usadas_grafo_sliding2)
    lista_usadas_grafo_sliding.append(lista_usadas_grafo_sliding2)

    fmr, fnmr, b_acc,  y_genuine , y_impostor = system.compute_metrics(y_true, y_pred)
    auxiliar_euclidiano2 = y_genuine.value_counts()[1], y_impostor.value_counts()[1]

    for met in ['fmr','fnmr','b_acc']:
        metrics_adaptativo_grafo_sliding.setdefault(genuine, dict()).setdefault(met,[]).append(eval(met))
    #json.dump(metrics_adaptativo_grafo_sliding, open("metricas_grafo_sliding.json", "w"))


usuarios = metrics_adaptativo_grafo_sliding.keys()
result = pd.DataFrame(metrics_adaptativo_grafo_sliding.values())

fmr_mean = []
fnmr_mean = []
b_acc_mean = []

for i in result['fmr']:
    fmr_mean.append(Average(i))

for i in result['fnmr']:
    fnmr_mean.append(Average(i))

for i in result['b_acc']:
    b_acc_mean.append(Average(i))

metrics_adaptativo_grafo_sliding_mean = pd.DataFrame(list(zip(usuarios, fmr_mean, fnmr_mean,b_acc_mean)),
            columns =['Usuarios','fmr_mean', 'fnmr_mean','b_acc_mean'])

GRAPH_MIN_CUT_SLIDING.append(metrics_adaptativo_grafo_sliding_mean['b_acc_mean'].mean())

#--------------------------------------------------------------------------------------------------------------#     

fim = timeit.default_timer()
horas, rem = divmod(fim-inicio, 3600)
minutos, segundos = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(horas),int(minutos),segundos))

print('---------------------')
print("Teste com Grafos distancia Euclidiana")
print("GRAPH_MIN_CUT_SLIDING", GRAPH_MIN_CUT_SLIDING)

Rodando GraphMinCutSliding
Testando usuário 51/51
---------------------
Teste com Grafos distancia Euclidiana
GRAPH_MIN_CUT_SLIDING [0.673426705882353]
